In [1]:
import re
from bs4 import BeautifulSoup
import requests
from lxml import etree

In [2]:
url = "https://terrikon.com/en/worldcup-2022"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}
r = requests.get(url,headers=headers)

In [3]:
soup = BeautifulSoup(r.text,"html.parser")
links = soup.find_all('td',class_ = 'team')

# print(re.compile(r'^<a.*?href="(.*)">').findall(r.text))
# print(re.compile(r'^<a.*?href="(.*)">').findall(str(links)))
# <td class="team"><a href="/en/football/teams/133">
# <img src="//st.terrikon.com/i/flag/s/nl.gif"/> Netherlands</a></td>
# re.compile(r'<a.*?href="(.*)">').findall(str(links))
link_list = []
for link in links:  
    if len(re.compile(r'<a.*?href="(.*)">').findall(str(link))) > 0:
        link_list.append((re.compile(r'<a.*?href="(.*)">').findall(str(link)))[0])
link_list = list(set(link_list))
link_list

['/en/football/teams/74',
 '/en/football/teams/531',
 '/en/football/teams/293',
 '/en/football/teams/1284',
 '/en/football/teams/878',
 '/en/football/teams/380',
 '/en/football/teams/133',
 '/en/football/teams/134',
 '/en/football/teams/228',
 '/en/football/teams/290',
 '/en/football/teams/263',
 '/en/football/teams/234',
 '/en/football/teams/260',
 '/en/football/teams/304',
 '/en/football/teams/320',
 '/en/football/teams/1281',
 '/en/football/teams/135',
 '/en/football/teams/289',
 '/en/football/teams/295',
 '/en/football/teams/301',
 '/en/football/teams/297',
 '/en/football/teams/553',
 '/en/football/teams/148',
 '/en/football/teams/349',
 '/en/football/teams/292',
 '/en/football/teams/1276',
 '/en/football/teams/131',
 '/en/football/teams/288',
 '/en/football/teams/275',
 '/en/football/teams/233',
 '/en/football/teams/296',
 '/en/football/teams/416']

In [8]:
players_list = []
for link in link_list:
    player_url = "https://terrikon.com/en/football/teams/" + re.findall(r"\d+",link)[0] +"/players"
    print(player_url)
    page_text = requests.get(player_url,headers=headers).text
    tree = etree.HTML(page_text)

    trlist = tree.xpath('/html/body/div[2]/div[2]/div[3]/div[3]/table/tr')

    for tr in trlist:
        player = {}
        player.setdefault("jersey_number",tr.xpath('./td')[0].text) #1
        if len(tr.xpath('./td[2]/img/@title')) >0:
            player.setdefault("nationality_name",tr.xpath('./td[2]/img/@title')[0]) #Англия
            temp = tr.xpath('./td[2]/img/@title')[0]
        else:
            player.setdefault("nationality_name",temp)
        player.setdefault("name",tr.xpath('./td[3]/a')[0].text) #Джордан Пикфорд
        player.setdefault("birth",tr.xpath('./td')[4].text)
        player.setdefault("high",tr.xpath('./td')[5].text)
        player.setdefault("weight",tr.xpath('./td')[6].text)
        players_list.append(player)
len(players_list)

https://terrikon.com/en/football/teams/74/players
https://terrikon.com/en/football/teams/531/players
https://terrikon.com/en/football/teams/293/players
https://terrikon.com/en/football/teams/1284/players
https://terrikon.com/en/football/teams/878/players
https://terrikon.com/en/football/teams/380/players
https://terrikon.com/en/football/teams/133/players
https://terrikon.com/en/football/teams/134/players
https://terrikon.com/en/football/teams/228/players
https://terrikon.com/en/football/teams/290/players
https://terrikon.com/en/football/teams/263/players
https://terrikon.com/en/football/teams/234/players
https://terrikon.com/en/football/teams/260/players
https://terrikon.com/en/football/teams/304/players
https://terrikon.com/en/football/teams/320/players
https://terrikon.com/en/football/teams/1281/players
https://terrikon.com/en/football/teams/135/players
https://terrikon.com/en/football/teams/289/players
https://terrikon.com/en/football/teams/295/players
https://terrikon.com/en/footba

780

In [9]:
players_list[1]

{'jersey_number': '16',
 'nationality_name': 'Denmark',
 'name': 'Oliver Christensen',
 'birth': '22.03.1999',
 'high': '190',
 'weight': '82'}

In [14]:
import csv
labels = list(player.keys())
try:
    with open('spider.csv', 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=labels)
        writer.writeheader()
        for elem in players_list:
            writer.writerow(elem)
except IOError:
    print("I/O error")

In [15]:
import pandas as pd
pd.read_csv('spider.csv')

,jersey_number,nationality_name,name,birth,high,weight
0,1,Denmark,Kasper Schmeichel,05.11.1986,185,76
1,16,Denmark,Oliver Christensen,22.03.1999,190,82
2,22,Denmark,Frederik Roennow,04.08.1992,190,79
3,2,Denmark,Joachim Andersen,31.05.1996,192,90
4,4,Denmark,Simon Kjaer,26.03.1989,190,86
...,...,...,...,...,...,...
775,23,Tunisia,Naïm Sliti,27.07.1992,173,75
776,25,Tunisia,Slimane A.,16.03.2001,188,80
777,9,Tunisia,Issam Jebali,25.12.1991,189,84
778,11,Tunisia,Khenissi,06.01.1992,183,0


In [12]:
from google.cloud import translate

# Instantiates a client
translate_client = translate.Client()

# The text to translate
text = u'Hello, world!'
# The target language
target = 'ru'

# Translates some text into Russian
translation = translate_client.translate(
    text,
    target_language=target)

print(u'Text: {}'.format(text))
print(u'Translation: {}'.format(translation['translatedText']))

ModuleNotFoundError: No module named 'google'